### 교차검증
- 부족한 데이터셋 및 특정 데이터에 과대적합되는 문제 해결을 위한 방안
- 학습 데이터셋을 일정 크기의 데이터 n개로 분리 후 1/n = 검증용, 나머지는 학습용으로 사용한다

1. 모듈 로딩, 데이터 준비
<hr>

In [42]:
import numpy as np
from sklearn.model_selection import KFold

X = np.array([[1,2], [3,4], [1,2], [3,4]])
y = np.array([1,2,3,4])

In [43]:
# kfold 인스턴스 생성: 데이터를 2개로 분할하는 객체

kfold = KFold(n_splits=2)


In [44]:
#데이터 분할하기
datasets = kfold.split(X, y)

In [45]:
for train, test in datasets:
    print(train, test)

[2 3] [0 1]
[0 1] [2 3]


In [46]:
import numpy as np
import pandas as pd

In [47]:
#Perch.csv file 5등분

perch = pd.read_csv('../data/perch3.csv')
perch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Weight  56 non-null     float64
 1   Length  56 non-null     float64
 2   Height  56 non-null     float64
 3   Width   56 non-null     float64
dtypes: float64(4)
memory usage: 1.9 KB


In [48]:
fold5 = KFold(n_splits=3)
datasets = fold5.split(perch)

for index, (train, test) in enumerate(datasets):
    print(f'{index}=>{train.shape},{test.shape}')

0=>(37,),(19,)
1=>(37,),(19,)
2=>(38,),(18,)


In [49]:
fold5 = KFold()
datasets = fold5.split(perch)

for index, (train, test) in enumerate(datasets):
    print(f'{index}=>{train.shape},{test.shape}')

0=>(44,),(12,)
1=>(45,),(11,)
2=>(45,),(11,)
3=>(45,),(11,)
4=>(45,),(11,)


In [50]:
# iris
# 타겟이 분류인 경우

iris = pd.read_csv('../data/iris.csv')

iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal.length  150 non-null    float64
 1   sepal.width   150 non-null    float64
 2   petal.length  150 non-null    float64
 3   petal.width   150 non-null    float64
 4   variety       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [51]:
k_fold = KFold(n_splits=3)
ret = k_fold.split(iris[iris.columns[:-1]])

In [52]:
for idx, (train, test) in enumerate(ret):
    print(idx, train, test)

0 [ 50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85
  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103
 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121
 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 146 147 148 149] [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49]
1 [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49 100 101 102 103
 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121
 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 146 147 148 149] [50 51 52 53 54 55 56 57 58 

In [61]:
from sklearn.linear_model import LogisticRegression

In [72]:
k_fold = KFold(n_splits=3, shuffle=True)
ret = k_fold.split(iris[iris.columns[:-1]])

trainscore3= []
for pname in ['l1','l2','elasticnet']:
    
    
    for idx, (train, test) in enumerate(ret):
        #학습용, 테스트용 인덱스 반환
        train_idx = train.tolist() #train = ndarray라서 리스트로 바까줌
        test_idx = test.tolist()
        
        #인덱스에 해당하는 데이터셋 추출하기
        trainDF = iris.iloc[train_idx]
        testDF = iris.iloc[test_idx]
        
        print(trainDF['variety'].value_counts()/trainDF.shape[0])
        print(testDF['variety'].value_counts()/testDF.shape[0])
        print('='*90)
        
        xtrain = trainDF[trainDF.columns[:-1]]
        ytrain = trainDF[trainDF.columns[-1]]
        xtest = testDF[testDF.columns[:-1]]
        ytest = testDF[testDF.columns[-1]]
        #분류모델 학습
        logmd = LogisticRegression(max_iter=1000, solver='liblinear', penalty= pname)
        logmd.fit(xtrain, ytrain)
        
        #훈련, 검증 성능
        trainscore = logmd.score(xtrain, ytrain)
        testscore = logmd.score(xtest, ytest)
        trainscore3.append(trainscore)
        
        #예측
        pre_y = logmd.predict(xtest)
        

variety
Versicolor    0.39
Virginica     0.32
Setosa        0.29
Name: count, dtype: float64
variety
Setosa        0.42
Virginica     0.36
Versicolor    0.22
Name: count, dtype: float64
variety
Setosa        0.35
Versicolor    0.33
Virginica     0.32
Name: count, dtype: float64
variety
Virginica     0.36
Versicolor    0.34
Setosa        0.30
Name: count, dtype: float64
variety
Setosa        0.36
Virginica     0.36
Versicolor    0.28
Name: count, dtype: float64
variety
Versicolor    0.44
Setosa        0.28
Virginica     0.28
Name: count, dtype: float64


In [69]:
sum(trainscore3)/3

0.96